In [5]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

1. Data Loader

In [2]:
class WeatherDataset(Dataset):
    def __init__(self, folder_path):
        self.labels = {'Sunny': 0, 'Snowy': 1, 'Cloudy': 2, 'Rainy': 3, 'Foggy': 4}
        self.data = []
        for file in os.listdir(folder_path):
            label_name = file.split('.')[0]  # 假设文件名的格式是'Cloudy1.jpg'
            for label in self.labels:
                if label in label_name:
                    self.data.append((os.path.join(folder_path, file), self.labels[label]))
                    break

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)
    
    def one_hot_encode(self, label, num_classes=5):
        return torch.eye(num_classes)[label]

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        image = self.transform(image)
        label = self.one_hot_encode(label)  # 将标签转换为one-hot编码
        return image, label


dataset = WeatherDataset(folder_path='../Data_Q2/train_data')
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [ ]:
model = models.vgg16()
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 5)  # 5个类别的输出

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(50):  # loop over the dataset multiple times
    epoch_loss = 0.0
    with tqdm(enumerate(dataloader, 0), total=len(dataloader), desc=f'Epoch {epoch+1}/{50}', unit='batch') as tepoch:
        for i, data in tepoch:
            inputs, labels = data
            labels = labels.float()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            tepoch.set_postfix(loss=epoch_loss/(i+1))


Epoch 4/50:  65%|██████▌   | 41/63 [00:29<00:15,  1.39batch/s, loss=0.496]

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in dataloader:
        images, labels = data
        labels = labels.float()
        outputs = model(images)
        predicted = outputs.sigmoid() > 0.5  # 应用sigmoid并将输出转换为二进制预测
        total += labels.numel()
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the network on the 250 images: {accuracy:.2f} %')
